In [1]:
import numpy as np
import pandas as pd

In [120]:
def read_to_list(path):
    out_list = []
    with open(path, 'r') as file:
        for line in file:
            out_list.append(line.rstrip())
    return out_list

def read_to_df(path):
    out_list = read_to_list(path)
    out_df = pd.DataFrame (out_list, columns=['strings'])
    return out_df

def clean_df(df_original):
    df = df_original.copy()
    df = df[df['strings'].str[0] != '#']
    df[['words','rest']] = df.strings.str.split('=',expand=True)
    df[['category', 'sources']] = df.rest.str.split(';fonts: ',expand=True)
    # df['category'] = df['category'].str.split('categories: ').apply(lambda x: x[1] if len(x) > 1 else None)
    df['category'] = df['category'].str.split('categories: ').apply(lambda x: None if x is None else (x[1] if len(x) > 1 else None))
    df.sources = df.sources.str[0:-1]
    df.drop(['strings','rest'],axis=1, inplace=True)
    return df

In [121]:
path = './Data/noms-fdic.txt'
noms_df = read_to_df(path)
noms_df = clean_df(noms_df)

path = './Data/adjectius-fdic.txt'
adj_df = read_to_df(path)
adj_df = clean_df(adj_df)

In [124]:
noms_df.head()

,words,category,sources
191,17-A,MS,data
192,17A,MS,data
193,4x4,MI,"esadir,termcat"
194,A3,MI,simbol
195,A4,MI,simbol
